# I. Pre-procesamiento de datos

Como disponemos de recursos computacionales limitados, adoptaremos una serie de supuestos razonables que nos permitan acotar el espacio de comparación, a la vez que se obtenga un resultado razonablemente bueno.

## Import y funciones de limpieza de String

In [7]:
### Correr estas dos líneas solo la primera vez si no has instalado nltk
#import nltk
#nltk.download('stopwords')
import pandas as pd
import numpy as np
import random
import re
from nltk.corpus import stopwords
import math
from tqdm import tqdm
from math import floor, comb

#https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python
def remove_emojis(text):
    """
    Devuelve texto (idealmente) sin emojis
    """
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', text)
stopwords = set(stopwords.words('spanish'))
#https://stackoverflow.com/questions/25346058/removing-list-of-words-from-a-string
def filter_sentence(text):
    querywords = text.split()
    resultwords  = [word for word in querywords if word.lower() not in stopwords]
    result = ' '.join(resultwords)
    return result

# Quitar caracteres raros
def remove_weird(text):
    return ''.join([c for c in text if ord(c) < 128])

# Aplicar todas las funciones anteriores
def string_clean_pipeline(text):
    text = remove_weird(text)
    text = remove_emojis(text)
    text = filter_sentence(text)
    return text

Función para crear funciones de hash tomada del material de clases

In [8]:
def crear_hash(a, b, p, n):
    def f(x):
        return ((a * x + b) % p) % n
    return f

## Lectura de datos

Se cargan los datos y muestran los usuarios únicos así como la cantidad de combinaciones de pares candidatos posibles.

In [9]:
df = pd.read_csv("tweets_2022_abril_junio.csv")
unicos = df.screen_name.unique()
print(f"Usuarios únicos: {len(unicos)}") 
print(f"Posibles combinaciones: {comb(len(unicos), 2)}")

Usuarios únicos: 208139
Posibles combinaciones: 21660817591


Conservaremos solo las columnas de interés

In [10]:
df = df.loc[:, ['id', 'screen_name', 'text']]

Primero quitaremos los retweets porque obviamente son muy similares entre ellos

In [11]:
resumen = df
resumen = resumen[~resumen['text'].str.lower().str.contains("rt @")] # quitar retweets
resumen = resumen.reset_index().iloc[:, 1:]
resumen

,id,screen_name,text
0,1512186668387913732,simonlodijo,@unveranonaranja @ruidosafest @franciscamusic ...
1,1512186985850544129,MacaSimplemente,@LaSuRivas @ElisaLoncon @siliconvalle @Valdebe...
2,1512187189974683661,LuisVer75699645,@teanval0207 @izkia @arturozunigaj Excelente...
3,1512187226398076929,MITERRED,@mcubillossigall https://t.co/gkg5PwbZhZ
4,1512187244248936452,piaelizabethvm,@simonlodijo @ruidosafest @franciscamusic @gio...
...,...,...,...
1266621,1525908559673712640,RichardDCorobo,@BottoConstituy1 El poder económico nunca va a...
1266622,1525981922115358720,AgainFlano,De @danielstingo no me lo esperaba ....\nEstim...
1266623,1526319160849932288,opinion_liberal,El @danielstingo es un #ConvencionalCSM !!!
1266624,1525857323691950080,aguja9999,¿y @Contraloriacl ?


Nos interesa gente que tiene más de 50 tweets, pues nos interesa acotar el costo computacional y es improbable que encontremos estilos similares entre dos autores que tienen menos tweets.

In [12]:
print("Forma antes de filtro", resumen.shape)
tweet_count = resumen.groupby('screen_name').agg({'screen_name':'count'}).rename({'screen_name':'n'}, axis = 1)
resumen = resumen[resumen.screen_name.isin(tweet_count[tweet_count.n > 50].index)]
print("Forma después de filtro", resumen.shape)

Forma antes de filtro (1266626, 3)
Forma después de filtro (553690, 3)


Eliminar tweets demasiado cortos

In [15]:
tweet_len = resumen.text.apply(len)
resumen = resumen[tweet_len>80]

Quitar emojis, caracteres exóticos y stopwords

In [17]:
resumen["clean_text"] = resumen["text"].apply(lambda x:  string_clean_pipeline(x))
### Agrupar tweets por persona y aglutinar todos sus tweet en una fila. No recomendado
# resumen = resumen.groupby('screen_name').agg({'clean_text':'sum', 'text':'sum', 'screen_name':'count'})
# resumen["shingles"] = resumen["clean_text"].apply(lambda x: get_shingles(x))
#resumen = resumen.rename({'screen_name':'tweet_count'}, axis = 1) 
#resumen = resumen[resumen.tweet_count > 40] # Conservar usuarios con mas de 40 tweets

## Shingling

Definimos trabajar con shingles con k=5

In [18]:
def get_shingles(text):
    k = 5
    result = set()
    #text = text.split(" ")
    for i in range(len(text) - k-1):
        shingle =  text[i:i+k] # By character
        #shingle = " ".join(text[i:i+k])# by trigram
        result.add(shingle)
    return result

Extraer shingles

In [19]:
resumen["shingles"] = resumen["clean_text"].apply(lambda x: get_shingles(x))
resumen.head()

,id,screen_name,text,clean_text,shingles
7,1512188055788027904,Rob92029357,@sokio @berfontaine No tienes como revatir alg...,@sokio @berfontaine revatir realidad expuesta ...,"{perso, sta p, ad ex, expu, real, ta pe, ine..."
20,1512193611575799812,AlejoLatigo,@Hugo_Gutierrez_ Yaaaaaa yyyyyyyyy????????????...,@Hugo_Gutierrez_ Yaaaaaa yyyyyyyyy????????????...,"{OCREC, ZOCRE, CRECE, ierre, rez_ , Serve, ECE..."
29,1512198930657955842,GordoUC_,@Molines01 @tere_marinovic Te están mostrando ...,@Molines01 @tere_marinovic estn mostrando rech...,"{. pue, nes01, terc, most, mbre., derec, ran..."
30,1512199583794835467,Orellana21Marce,@mdaza_abogado @Jess99041498 Es que la derecha...,@mdaza_abogado @Jess99041498 derecha quiere em...,"{adrn , decen, 90414, 41498, @Jes, ladr, go..."
50,1512212224613359617,PATRIOTCHILENO,@tere_marinovic @Sebasti45781359 Orsoni cero a...,@tere_marinovic @Sebasti45781359 Orsoni cero a...,"{onta , 59 Or, 57813, 1359 , o apo, r mas, @Ma..."


Generar Vocabulario de Shingles a partir de su unión y mostrar su largo

In [31]:
d = resumen.loc[:, 'shingles']
un = set().union(*d)
len(un)

997134

El vocabulario es demasiado largo para nuestros propósitos. Procedemos a contar cuantas en cuantos tweet aparece cada shingle.

In [32]:
shingle_count = dict()
for row in tqdm(d):
    for item_set in row:
        try:
            shingle_count[item_set] += 1
        except KeyError:
            shingle_count[item_set] = 1

100%|██████████| 203726/203726 [00:03<00:00, 56593.93it/s]


Ahora conservamos los shingles que aparecen en al menos 230 tweets, para quedarnos con un vocabulario de alrededor de 10.000 shingles

In [39]:
un_sample = [key for key, value in shingle_count.items() if value > 230] # smpling by freq
print(len(un_sample))

9958


De esta forma, obtenemos un vocabulario más acotado que acorta los cómputos. Procedemos a guardar los shingles en un archivo de texto

In [40]:
#un_sample = set(random.sample(un, 50000)) # random
with open("shingles_sample.txt", 'w') as file:
    file.write(str(un_sample))

Ahora convertimos la columna de shingles en tuplas para que sean más fáciles de guardar y releer desde un .csv

In [41]:
resumen.shingles = resumen.shingles.apply(lambda x: tuple(x))
resumen.to_csv("preprocesado.csv")

Con esto terminamos con el preprocesamiento del archivo de tweets.

In [ ]:
resumen